# Задача 5. Экспериментальное исследование алгоритмов для регулярных запросов
# Автор: [Miroshnikov Vladislav](https://github.com/vladislav-miroshnikov)
## Введение

Задача посвящена анализу производительности различных реализаций алгоритма решения задачи достижимости между всеми парами вершин с регулярными ограничениями через тензорное произведение.

Перейдем к постановке задачи:

В задаче имеется размеченный граф $G$, допускающий регулярный язык $L_G$

Также имеется регулярное выражение $R$, которое задает регулярный язык $L_R$, который мы считаем нашим языком ограничений.

Рассмотрим задачу $RPQ = \{(v_i, v_j)|\exists\pi:w(v_i\pi v_j)\in L, v_i\in V_S, v_j\in V_F\}$, где $L = L_G\cap L_R$

Чтобы построить язык пересечения, нужно построить пересечение конечных автоматов $КА_{G}$ и $КА_{R}$.

Введем определение пересечения конечных автоматов:

$КА_3 = КА_1\cap КА_2 = (S^{1}\times S^{2}, \Delta^{3}, S^{1}_S\times S^{2}_S, S^{1}_F\times S^{2}_F)$, где функция переходов задаётся как
$\Delta^{3}: (v_i, v_j)\times l\rightarrow (u_i, u_j)$

$
\begin{cases}
v_i\in S^{1}\times l\rightarrow u_i\in S^{1}\quad\in\Delta^{1} \\
v_j\in S^{2}\times l\rightarrow u_j\in S^{2}\quad\in\Delta^{2}
\end{cases}$

Построение функции переходов осуществляется через тензорное произведении матриц смежностей конечных автоматов. 

## Постановка задачи
### Цели и задачи
Целью данного исследования является проведение эксперимента, основанного на осуществлении регулярных запросов к набору графов в двух вариантах реализации алгоритма пересечения конечных автоматов с дальнейшим анализом полученных результатов.

Для достижения цели необходимо:
- Используя библиотеку pyCuBool, основанную на технологии NVIDIA CUDA,  реализовать алгоритм пересечения двух конечных автоматов через тензорное произведение
- Взять ранее реализованный алгоритм для пересечения двух конечных автоматов через тензорное произведение
- Сформировать и описать набор данных для экспериментов
- Сформулировать методологию проведения эксперимента
- Произвести сравнительный анализ производительности реализаций алгоритмов


## Исследуемые решения
Элементами матрицы смежности конечного автомата являются подмножества его меток. Чтобы не вводить операцию пересечения множеств в тензорном произведении, мы будем приводить матрицы смежности двух конечных автоматов к виду булевых матриц.

Булевы матрицы на практике являются сильно разреженными. Для работы с подобными разреженными матрицами мы будем использовать алгоритм в двух вариантах реализации, основанный на следующих библиотеках: 
- [puCuBool](https://pypi.org/project/pycubool/)
- [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html)


## Настройка и инициализация рабочего окружения

Установка библиотеки pycubool

In [1]:
!pip install pycubool

Скачивание и установка проекта https://github.com/vladislav-miroshnikov/formal-lang-course

In [2]:
!git clone https://github.com/vladislav-miroshnikov/formal-lang-course

fatal: destination path 'formal-lang-course' already exists and is not an empty directory.


Установка зависимостей проекта

In [3]:
!pip install --upgrade pip
!pip install -r formal-lang-course/requirements.txt

Подключение директории проекта

In [4]:
import sys
sys.path.insert(1, 'formal-lang-course')

## Описание данных для экспериментов
### Графы

В качестве $L_G$ будем использовать графы из [RDF](https://jetbrains-research.github.io/CFPQ_Data/dataset/RDF.html) датасета.

Приведем информаци о графах, которые будут использоваться в задаче:

In [5]:
GRAPHS = (
    "skos",
    "generations",
    "travel",
    "univ_bench",
    "atom_primitive",
    "biomedical_mesure_primitive",
    "foaf",
    "people_pets",
    "funding",
    "wine",
    "pizza",
    "core",
    "pathways",
    "enzyme",
    #"eclass_514en",
    #"go_hierarchy",
    #"go",
    #"geospecies",
)

**Пределы возможностей**: 

Стоит отметить, что в данный список не вошли графы taxonomy, taxonomy_hierarchy, поскольку вычислительных возможностей Google Colab - среды выполнения, недостаточно для обработки этих графов.

In [6]:
from project import get_graph_info_util

for graph in GRAPHS:
  print(f"'{graph}': ")
  get_graph_info_util(graph)

'skos': 
Information about graph:

        Count of nodes: 144
        Count of edges: 252
        Labels: {rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#range'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subClassOf'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#label'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#first'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#domain'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#type'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#subPropertyOf'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#scopeNote'), rdflib.term.URIRef('http://www.w3.org/2004/02/skos/core#definition'), rdflib.term.URIRef('http://purl.org/dc/terms/contributor'), rdflib.term.URIRef('http://purl.org/dc/terms/creator'), rdflib.term.URIRef('http://www.w3.org/2000/01/rdf-schema#seeAlso'), rdflib.term.URIRef('http://www.w3.org/1999/02/22-rdf-syntax-ns#rest'), rdflib.term

## Регулярные запросы

Ниже приведены регулярные запросы, которые будут производится к наборам наших графов. 

Данные запросы используют все общепринятые конструкции регулярных выражений (замыкание, конкатенация, звезда Клини):



1.   l1 (l2 | l3)* 
2.   l1* (l2 | l3)*
3.   l0 l1 l2 (l3 | l1)*
4.   (l0 | l3)* | (l1 | l2)*



In [7]:
from pyformlang.regular_expression.regex_objects import Symbol

def regex_from_label(label):
  regex = Regex("")
  regex.head = Symbol(str(label))
  return regex

def regex_first(labels):
  """
  l1 (l2 | l3)*
  """
  regex_1 = regex_from_label(labels[0])
  regex_2 = regex_from_label(labels[1])
  regex_3 = regex_from_label(labels[2])
  return regex_1.concatenate((regex_2.union(regex_3)).kleene_star())

def regex_second(labels):
  """
   l1* (l2 | l3)*
  """
  regex_1 = regex_from_label(labels[0])
  regex_2 = regex_from_label(labels[1])
  regex_3 = regex_from_label(labels[2])
  return regex_1.kleene_star().concatenate((regex_2.union(regex_3).kleene_star()))

def regex_third(labels):
  """
  l0 l1 l2 (l3 | l1)*
  """ 
  regex_0 = regex_from_label(labels[0])
  regex_1 = regex_from_label(labels[1])
  regex_2 = regex_from_label(labels[2])
  regex_3 = regex_from_label(labels[3])
  return regex_0.concatenate(regex_1).concatenate(regex_2).concatenate((regex_3.union(regex_1)).kleene_star())

def regex_fourth(labels):
  """
  (l0 | l3)* | (l1 | l2)*
  """
  regex_0 = regex_from_label(labels[0])
  regex_1 = regex_from_label(labels[1])
  regex_2 = regex_from_label(labels[2])
  regex_3 = regex_from_label(labels[3])
  left_regex = (regex_0.union(regex_3)).kleene_star()
  right_regex = (regex_1.union(regex_2)).kleene_star()
  return left_regex.union(right_regex)

### Класс для булевых матриц 
Данный класс для булевых матрицы использует библиотеку **pycubool**. 


In [8]:
from pyformlang.finite_automaton import NondeterministicFiniteAutomaton
import pycubool as cb

__all__ = ["CuBooleanMatrices"]

#Based on gpu library
class CuBooleanMatrices:
    """
    Representation of NFA as a Boolean Matrix

    Attributes
    ----------
    states_count: set
        Count of states
    state_indices: dict
        Dictionary of states
    start_states: set
        Start states of NFA
    final_states: set
        Final states of NFA
    bool_matrices: dict
        Dictionary of boolean matrices.
        Keys are NFA symbols
    """

    def __init__(self, n_automaton: NondeterministicFiniteAutomaton = None):
        if n_automaton is None:
            self.states_count = 0
            self.state_indices = dict()
            self.start_states = set()
            self.final_states = set()
            self.bool_matrices = dict()
        else:
            self.states_count = len(n_automaton.states)
            self.state_indices = {
                state: index for index, state in enumerate(n_automaton.states)
            }
            self.start_states = n_automaton.start_states
            self.final_states = n_automaton.final_states
            self.bool_matrices = self.init_bool_matrices(n_automaton)

    def get_states(self):
        return self.state_indices.keys()

    def get_start_states(self):
        return self.start_states

    def get_final_states(self):
        return self.final_states

    def init_bool_matrices(self, n_automaton: NondeterministicFiniteAutomaton):
        """
        Initialize boolean matrices of NondeterministicFiniteAutomaton

        Parameters
        ----------
        n_automaton: NondeterministicFiniteAutomaton
            NFA to transform to matrix

        Returns
        -------
        bool_matrices: dict
            Dict of boolean matrix for every automata label-key
        """
        bool_matrices = dict()
        nfa_dict = n_automaton.to_dict()
        for state_from, trans in nfa_dict.items():
            for label, states_to in trans.items():
                if not isinstance(states_to, set):
                    states_to = {states_to}
                for state_to in states_to:
                    index_from = self.state_indices[state_from]
                    index_to = self.state_indices[state_to]
                    if label not in bool_matrices:
                        bool_matrices[label] = cb.Matrix.empty(shape=(self.states_count, self.states_count))           
                    bool_matrices[label][index_from, index_to] = True

        return bool_matrices

    def make_transitive_closure(self):
        """
        Makes transitive closure of boolean matrices

        Returns
        -------
        tc: cuBool matrix
            Transitive closure of boolean matrices
        """
        if not self.bool_matrices.values():
            return cb.Matrix.empty(shape=(2, 2))

        shape = list(self.bool_matrices.values())[0].shape
        tc = cb.Matrix.empty(shape=shape)

        for elem in self.bool_matrices.values():
            tc = tc.ewiseadd(elem)
        prev_nnz = tc.nvals
        curr_nnz = 0

        while prev_nnz != curr_nnz:
            tc = tc.ewiseadd(tc.mxm(tc))
            prev_nnz, curr_nnz = curr_nnz, tc.nvals


        return tc


### Дополнительные функции 
Введем дополнительные функции для работы с классом булевых матриц, объявленных выше

In [9]:
from pyformlang.finite_automaton import NondeterministicFiniteAutomaton, State
import pycubool as cb

__all__ = ["intersect_boolean_matrices", "convert_bm_to_automaton"]


def intersect_boolean_matrices(self: CuBooleanMatrices, other: CuBooleanMatrices):
    """
    Makes intersection of self boolean matrix with other

    Parameters
    ----------
    self: CuBooleanMatrices
        Left-hand side boolean matrix
    other: CuBooleanMatrices
        Right-hand side boolean matrix

    Returns
    -------
    intersect_bm: CuBooleanMatrices
        Intersection of two boolean matrices
    """
    intersect_bm = CuBooleanMatrices()
    intersect_bm.num_states = self.states_count * other.states_count
    common_symbols = self.bool_matrices.keys() & other.bool_matrices.keys()

    for symbol in common_symbols:
        intersect_bm.bool_matrices[symbol] = self.bool_matrices[symbol].kronecker(other.bool_matrices[symbol])
        
    for state_fst, state_fst_index in self.state_indices.items():
        for state_snd, state_snd_idx in other.state_indices.items():
            new_state = new_state_idx = (
                state_fst_index * other.states_count + state_snd_idx
            )
            intersect_bm.state_indices[new_state] = new_state_idx

            if state_fst in self.start_states and state_snd in other.start_states:
                intersect_bm.start_states.add(new_state)

            if state_fst in self.final_states and state_snd in other.final_states:
                intersect_bm.final_states.add(new_state)

    return intersect_bm


def convert_bm_to_automaton(boolean_matrices: CuBooleanMatrices):
    """
    Converts CuBooleanMatrices to NFA

    Returns
    -------
    automaton: NondeterministicFiniteAutomaton
        Representation of CuBooleanMatrices as NFA
    """
    automaton = NondeterministicFiniteAutomaton()
    for label, bool_matrix in boolean_matrices.bool_matrices.items():
        for state_from, state_to in zip(*bool_matrix.nonzero()):
            automaton.add_transition(state_from, label, state_to)

    for state in boolean_matrices.start_states:
        automaton.add_start_state(State(state))

    for state in boolean_matrices.final_states:
        automaton.add_final_state(State(state))

    return automaton


Функция, предназначенная для выполнения регулярных запросов

In [35]:
import networkx as nx
from project import get_nfa_by_graph, regex_to_min_dfa
from pyformlang.regular_expression import Regex

__all__ = ["cu_bool_rpq"]


def cu_bool_rpq(
    graph: nx.MultiDiGraph,
    query: Regex,
    start_nodes: set = None,
    final_nodes: set = None,
) -> set:
    """
    Computes Regular Path Querying from given graph and regular expression

    Parameters
    ----------
    graph: MultiDiGraph
       Labeled graph
    query: str
       Regular expression given as string
    start_nodes: set, default=None
       Start states in NFA
    final_nodes: set, default=None
       Final states in NFA

    Returns
    -------
    result_set: set
       Regular Path Querying
    """
    nfa_by_graph = get_nfa_by_graph(graph, start_nodes, final_nodes)
    dfa_by_graph = query.to_epsilon_nfa().minimize()

    graph_bm = CuBooleanMatrices(nfa_by_graph)
    query_bm = CuBooleanMatrices(dfa_by_graph)

    intersected_bm = intersect_boolean_matrices(graph_bm, query_bm)
    transitive_closure = intersected_bm.make_transitive_closure()

    start_states = intersected_bm.get_start_states()
    final_states = intersected_bm.get_final_states()

    result_set = set()

    for state_from, state_to in transitive_closure.to_list():
        if state_from in start_states and state_to in final_states:
            result_set.add(
                (state_from // query_bm.states_count, state_to // query_bm.states_count)
            )

    return result_set


In [31]:
from project import rpq
from cfpq_data import graph_from_dataset, get_labels

print("--------- ALGORITHM BASED ON scipy.sparse LIBRARY ---------")

REGULAR_QUERIES = (regex_first, regex_second, regex_third, regex_fourth)

for graph_name in GRAPHS:
  graph = graph_from_dataset(graph_name, verbose=False)
  labels = list(get_labels(graph, verbose=False))

  for query in REGULAR_QUERIES:
    regular_expression = query(labels)
    print(f"\n  Graph: {graph_name}; \n  Regex: {query.__doc__}Result: ", end='')
    %timeit -n 5 result = rpq(graph, str(regular_expression))

--------- ALGORITHM BASED ON scipy.sparse LIBRARY ---------

  Graph: skos; 
  Regex: 
  l1 (l2 | l3)*
  Result: 5 loops, best of 5: 27.6 ms per loop

  Graph: skos; 
  Regex: 
   l1* (l2 | l3)*
  Result: 5 loops, best of 5: 26.7 ms per loop

  Graph: skos; 
  Regex: 
  l0 l1 l2 (l3 | l1)*
  Result: 5 loops, best of 5: 29.4 ms per loop

  Graph: skos; 
  Regex: 
  (l0 | l3)* | (l1 | l2)*
  Result: 5 loops, best of 5: 30 ms per loop

  Graph: generations; 
  Regex: 
  l1 (l2 | l3)*
  Result: 5 loops, best of 5: 29.6 ms per loop

  Graph: generations; 
  Regex: 
   l1* (l2 | l3)*
  Result: 5 loops, best of 5: 30.1 ms per loop

  Graph: generations; 
  Regex: 
  l0 l1 l2 (l3 | l1)*
  Result: 5 loops, best of 5: 34.3 ms per loop

  Graph: generations; 
  Regex: 
  (l0 | l3)* | (l1 | l2)*
  Result: 5 loops, best of 5: 32.3 ms per loop

  Graph: travel; 
  Regex: 
  l1 (l2 | l3)*
  Result: 5 loops, best of 5: 28.8 ms per loop

  Graph: travel; 
  Regex: 
   l1* (l2 | l3)*
  Result: 5 loops, 

In [36]:
from cfpq_data import graph_from_dataset, get_labels

print("--------- ALGORITHM BASED ON pycubool LIBRARY ---------")

REGULAR_QUERIES = (regex_first, regex_second, regex_third, regex_fourth)

for graph_name in GRAPHS:
  graph = graph_from_dataset(graph_name, verbose=False)
  labels = list(get_labels(graph, verbose=False))

  for query in REGULAR_QUERIES:
    regular_expression = query(labels)
    print(f"\n  Graph: {graph_name}; \n  Regex: {query.__doc__}Result: ", end='')
    %timeit -n 5 result = cu_bool_rpq(graph, regular_expression)

--------- ALGORITHM BASED ON pycubool LIBRARY ---------

  Graph: skos; 
  Regex: 
  l1 (l2 | l3)*
  Result: 5 loops, best of 5: 20.6 ms per loop

  Graph: skos; 
  Regex: 
   l1* (l2 | l3)*
  Result: 5 loops, best of 5: 20.8 ms per loop

  Graph: skos; 
  Regex: 
  l0 l1 l2 (l3 | l1)*
  Result: 5 loops, best of 5: 21.9 ms per loop

  Graph: skos; 
  Regex: 
  (l0 | l3)* | (l1 | l2)*
  Result: 5 loops, best of 5: 21.1 ms per loop

  Graph: generations; 
  Regex: 
  l1 (l2 | l3)*
  Result: 5 loops, best of 5: 22.6 ms per loop

  Graph: generations; 
  Regex: 
   l1* (l2 | l3)*
  Result: 5 loops, best of 5: 23 ms per loop

  Graph: generations; 
  Regex: 
  l0 l1 l2 (l3 | l1)*
  Result: 5 loops, best of 5: 22.2 ms per loop

  Graph: generations; 
  Regex: 
  (l0 | l3)* | (l1 | l2)*
  Result: 5 loops, best of 5: 23 ms per loop

  Graph: travel; 
  Regex: 
  l1 (l2 | l3)*
  Result: 5 loops, best of 5: 22.4 ms per loop

  Graph: travel; 
  Regex: 
   l1* (l2 | l3)*
  Result: 5 loops, best o

## Методология проведения эксперимента
Для сравнения производительности алгоритма к каждому графу создается по 4 регулярных запроса, каждый запрос выполняется на графе 5 раз. 

Приведём в таблице результаты работы алгоритма в зависимости от графа и созданного запроса.

В эксперименте оцениваются следующий показатель: 
*   лучший результат из 5 запросов



**Cреда выполнения:** Google Colab с аппаратным ускорителем GPU. 

### Полученные результаты

#### **Алгоритм, основанный на scipy.sparse**
##### Время работы алгоритма (в мс) в зависимости от графа и регулярного выражения:


|      regex on graph           | skos  | generations | travel | univ_bench | atom_primitive | biomedical_mesure_primitive | foaf | people_pets | funding | wine | pizza | core  | pathways | enzyme |
|----------------------------|-------|-------------|--------|------------|-------|-----------|--------|--------|--------|--------|--------|--------|--------|--------|
| l1 (l2 \| l3)*            | 27.6 |     29.6      |  28.8   | 31.3   |  47.8   | 51  |  58.7    | 65.5  | 117 |181 |181 |281 |1250 |8410
| l1* (l2 \| l3)*            |  26.7|    30.1       |  29.8   | 32.1   | 48.3    | 53.6  |  60.3    | 69.4  |120 |179 |180 |283 |1250|8450
| l0 l1 l2 (l3 \| l1)*            |  29.4|     34.3      |  32.2   |  35.3  |  51.3   | 60.2  |   64.5   | 72.1  |126 |189 | 189 |292  |1290|8880
| (l0 \| l3)* \| (l1 \| l2)*            | 30 |     32.3      |  31.4   | 34.9   |  51.8   | 55.1  |   64.4   | 71.2  |126 |186 |185 |294 |1250|8370
 


#### **Алгоритм, основанный на pycubool**
##### Время работы алгоритма (в мс) в зависимости от графа и регулярного выражения:


|      regex on graph           | skos  | generations | travel | univ_bench | atom_primitive | biomedical_mesure_primitive | foaf | people_pets | funding | wine | pizza | core  | pathways | enzyme |
|----------------------------|-------|-------------|--------|------------|-------|-----------|--------|--------|--------|--------|--------|--------|--------|--------|
| l1 (l2 \| l3)*            | 20.6 |     22.6       |  22.4    | 24.8    |  37    | 41.1   |  45.3     | 51.5   | 87.8  |140  |142  |228  |1030 |7020
| l1* (l2 \| l3)*            |  20.8|    23        | 22.1    | 24.9    | 37.3     | 39.8   |  46.3     | 51.4   |92.8  |144  |142  |227 |1020|6980
| l0 l1 l2 (l3 \| l1)*            |  21.9 |     22.2       |  23    |  24.9  |  38.1   | 41.6   |   46.4    | 52   |92.5  |146  | 146  |232   |1030|7070
| (l0 \| l3)* \| (l1 \| l2)*            | 21.1  |     23       |  23    | 25.3    |  37.9    | 41.6   |   46.8    | 51.8   |94.6  |144  |143  |230 |1030|7100
 

## Анализ полученных результатов

Исходя из полученных данных и построенных таблиц, видно, что алгоритм, реализованный с использованием библиотеки **pycubool** показал лучшие результаты, нежели алгоритм, реализованный с использованием библиотеки **scipy.sparse**. Примечательно также, что "выигрыш", то есть разница по времени между двумя алгоритма в процентном соотношении составляет примерно 20%.

### Заключение

Как было уставновлено выше, алгоритм решения задачи $RPQ$ показал лучшие результаты в случае применения библиотеки *pycubool* для реализации операций с булевыми матрицами. Результат можно объяснить тем, что библиотека *pycubool* во внутренней реализации использует возможности *NVIDIA CUDA*. Цель исследования успешно достигнута.
